## Instituto Federal da Paraíba
## Disciplina: Mineração de Dados
## Professor: Paulo Ribeiro
## Alunos: Filipe Salomão, Hércules de Sousa e Matheus Alves
##
## Projeto: Classificação de Gêneros Musicais

Aqui importamos a biblioteca Librosa. Essa biblioteca é uma ferramenta de processamento de áudio em Python. Ela fornece uma série de funções e utilitários para análise de áudio e extração de características de arquivos de áudio. É uma biblioteca amplamente utilizada em tarefas de processamento de áudio, como análise de música, reconhecimento de fala e aplicações de aprendizado de máquina relacionadas ao áudio.

In [1]:
import itertools
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.stats import kurtosis
from scipy.stats import skew

Este código define uma função chamada **_get_features_** que recebe um sinal de áudio **_y_**, uma taxa de amostragem **_sr_** e alguns parâmetros opcionais relacionados à análise de áudio, como o tamanho da janela de transformação de Fourier **_(n_fft)_** e o tamanho do salto entre as janelas **_(hop_length)_**.

A função tem como objetivo extrair várias características do sinal de áudio usando a biblioteca Librosa e retornar essas características como um dicionário.

Várias funções do Librosa são chamadas para extrair características específicas do sinal de áudio, como centro espectral, roll-off espectral, taxa de cruzamento por zero, RMS, força de início, contraste espectral, largura de banda espectral e planicidade espectral. O resultado de cada função é uma matriz multidimensional que é convertida em um vetor unidimensional usando o método **_.ravel_**.

Também as características MFCC (Mel-frequency Cepstral Coefficients) são calculadas usando a função **_librosa.feature.mfcc_**.

Uma função interna chamada **_get_moments_** é definida. Essa função calcula várias estatísticas descritivas para cada descritor, como valor máximo, valor mínimo, média, desvio padrão, curtose e assimetria.

In [2]:
def get_features(y, sr, n_fft=1024, hop_length=512):
    features = {}

    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_rolloff'] = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zero_crossing_rate'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rms'] = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['onset_strength'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['spectral_contrast'] = librosa.feature.spectral_contrast(y=y, sr=sr).ravel()
    features['spectral_bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_flatness'] = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length).ravel()

    mfcc = librosa.feature.mfcc(y=y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)

    for index, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(index)] = v_mfcc.ravel()

    def get_moments(descriptors):
        result = {}

        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)

        return result

    additional_features = get_moments(features)
    additional_features['tempo'] = librosa.feature.tempo(y=y, sr=sr)[0]

    return additional_features

A função a seguir recebe dois argumentos: **_genres_directory_**, que é o diretório principal contendo subdiretórios para diferentes gêneros musicais, e **_genres_**, que é uma lista dos gêneros a serem processados.

A função tem como objetivo percorrer os arquivos de áudio em cada subdiretório de gênero e extrair características desses arquivos usando a função **_get_features_** definida acima.

In [3]:
def process_genres_directory_files(genres_directory, genres):
    all_features = []

    for genre in genres:
        genre_directory = f'{genres_directory}/{genre}/wav_files'

        for file in os.listdir(genre_directory):
            file_path = f"{genre_directory}/{file}"

            y, sr = librosa.load(file_path)

            features = get_features(y, sr)

            features['label'] = genre

            all_features.append(features)

    return all_features

Diretório de previews:

In [4]:
genres_directory = './previews'

Lista de gêneros contidos no diretório:

In [5]:
genres = os.listdir(genres_directory)

In [6]:
genres

['forró', 'funk', 'jazz', 'rap', 'rock', 'samba', 'sertanejo']

Processando todos os arquivos de áudio contidos nos subdiretórios:

In [7]:
%%time

all_features = process_genres_directory_files(genres_directory, genres)

CPU times: total: 10min 49s
Wall time: 13min 38s


Transformando as características em um DataFrame:

In [8]:
df_features = pd.DataFrame(all_features)

In [9]:
df_features.shape

(840, 128)

In [10]:
df_features.head()

,spectral_centroid_max,spectral_centroid_min,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_kurtosis,spectral_centroid_skew,spectral_rolloff_max,spectral_rolloff_min,spectral_rolloff_mean,spectral_rolloff_std,...,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo,label
0,4737.508153,635.664676,2098.726133,556.496663,2.887029,1.283600,9130.078125,925.927734,4514.561770,1372.578815,...,-0.086715,-0.088010,32.658615,-31.948511,-5.576648,9.920987,-0.200280,0.234563,143.554688,forró
1,5339.076724,1019.369702,2294.895607,604.767987,1.803070,0.943630,9130.078125,1722.656250,4914.999411,1341.457851,...,0.120776,-0.207388,21.607838,-36.618702,-4.257775,9.034795,-0.080838,0.118976,143.554688,forró
2,6613.348219,1526.374864,2857.957595,685.797071,3.499504,1.245568,9474.609375,2648.583984,6027.380220,1304.601244,...,-0.253195,0.186958,32.782486,-35.293335,-7.233448,11.249509,-0.106279,0.236294,151.999081,forró
3,6935.364306,945.116845,2439.278264,730.595169,4.671441,1.223784,9840.673828,1184.326172,5551.766429,1564.960933,...,0.052664,-0.347261,28.608442,-36.924839,-3.531663,9.956684,0.135261,-0.020543,107.666016,forró
4,7086.838756,928.772862,2541.888664,744.284935,2.716824,1.082830,9969.873047,1162.792969,6118.729668,1422.390345,...,-0.254407,-0.014688,29.134048,-33.681553,-3.894992,9.132825,0.035545,0.014757,135.999178,forró


## Sobre as características do sinal de áudio

### Centroide espectral

A centroide espectral é uma medida que representa a posição média do espectro de frequência de um sinal de áudio. Em outras palavras, ela indica a frequência central em torno da qual a energia espectral está distribuída.

**Exemplos em gêneros musicais:**

No forró, a centroide espectral pode ser mais baixa, indicando uma concentração de energia em frequências mais baixas, típicas dos instrumentos como o acordeão e o zabumba.

No funk, a centroide espectral pode ser mais alta, indicando uma concentração de energia em frequências médias e altas, características dos graves e sons eletrônicos presentes no gênero.

No jazz, a centroide espectral pode ser moderada, refletindo uma distribuição equilibrada de energia em várias faixas de frequência, com destaque para os instrumentos como saxofone, piano e contrabaixo.

No rap, a centroide espectral pode ser mais baixa, enfatizando a presença de batidas graves e vocais em frequências médias.

No rock, a centroide espectral pode ser mais alta, destacando guitarras distorcidas, bateria e vocais que ocupam uma ampla faixa de frequência.

No samba, a centroide espectral pode ser moderada, com destaque para percussão, instrumentos de sopro e vocais em frequências médias.

No sertanejo, a centroide espectral pode ser mais baixa, enfatizando instrumentos como violão, acordeão e vocais em frequências médias e baixas.

### Rolloff

O rolloff espectral é definido como a frequência abaixo da qual uma certa porcentagem (geralmente 85%) da energia total do sinal de áudio está contida nas componentes de frequência mais baixas. Essa medida é útil na análise de áudio, pois pode ser usada para inferir informações sobre a "brilho" ou "timbre" do som.

**Exemplos em gêneros musicais:**

No forró, o rolloff pode ser mais baixo, indicando que a energia está concentrada em frequências mais baixas, como o grave do acordeão e do zabumba.

No funk, o rolloff pode ser mais alto, indicando uma concentração de energia em frequências médias e altas, como os agudos dos instrumentos de percussão e os sons eletrônicos.

No jazz, o rolloff pode estar em uma faixa intermediária, indicando uma distribuição equilibrada de energia em várias faixas de frequência, destacando os instrumentos como saxofone, piano e contrabaixo.

No rap, o rolloff pode ser mais baixo, enfatizando a presença de batidas graves e vocais em frequências médias.

No rock, o rolloff pode ser mais alto, destacando as guitarras distorcidas, bateria e vocais que ocupam uma ampla faixa de frequência.

No samba, o rolloff pode estar em uma faixa intermediária, com destaque para a percussão, instrumentos de sopro e vocais em frequências médias.

No sertanejo, o rolloff pode ser mais baixo, enfatizando instrumentos como violão, acordeão e vocais em frequências médias e baixas.

### Taxa de cruzamento por zero

A taxa de cruzamento por zero é uma medida que indica a quantidade de vezes que o sinal atravessa o eixo horizontal (valor zero) em relação ao tempo. Ela pode ser muito útil ao indicar informações sobre "aspereza" ou "textura".

**Exemplos em gêneros musicais:**

No forró, que geralmente possui ritmos marcados por instrumentos percussivos e acordeão, a taxa de cruzamento por zero pode ser alta, devido às rápidas mudanças de polaridade nos sons percussivos e nas notas do acordeão.

No funk, conhecido por suas batidas fortes e graves, a taxa de cruzamento por zero pode ser moderada a alta, devido às variações intensas dos graves e aos efeitos sonoros eletrônicos que possuem transições rápidas entre polaridades.

No jazz, que engloba uma variedade de instrumentos e estilos, a taxa de cruzamento por zero pode variar dependendo da música. Em partes mais calmas, como um solo de saxofone, a taxa pode ser baixa, enquanto em partes mais rítmicas, como uma seção de bateria, a taxa pode ser mais alta.

No rap, onde a ênfase está nas letras e nas batidas de fundo, a taxa de cruzamento por zero pode ser moderada, devido às batidas percussivas e aos vocais com mudanças de polaridade.

No rock, com suas guitarras distorcidas e bateria intensa, a taxa de cruzamento por zero pode ser alta, devido às transições rápidas e agressivas do sinal de áudio.

No samba, um estilo musical cheio de ritmo e percussão, a taxa de cruzamento por zero pode ser moderada a alta, devido às variações rítmicas das batidas e dos instrumentos percussivos.

No sertanejo, com sua instrumentação que inclui violão e acordeão, a taxa de cruzamento por zero pode ser moderada, dependendo do ritmo e da intensidade do arranjo.

### RMS

O RMS (Root Mean Square) é uma medida que calcula a média quadrática das amplitudes de um sinal de áudio. Basicamente, o RMS nos dá uma ideia de quão alto ou suave um sinal de áudio é em termos de intensidade.

**Exemplos em gêneros musicais:**

No forró, um estilo musical alegre e dançante, o RMS pode ser moderado a alto, devido à presença de instrumentos como o acordeão, que produz notas com amplitudes variadas.

No funk, com suas batidas intensas e graves marcantes, o RMS pode ser alto, devido à presença de linhas de baixo proeminentes e sons percussivos com amplitudes elevadas.

No jazz, que possui uma ampla variedade de estilos e instrumentações, o RMS pode variar dependendo da música. Em partes calmas, como um solo de piano suave, o RMS pode ser baixo, enquanto em partes mais animadas, como um solo de trompete, o RMS pode ser mais elevado.

No rap, onde a ênfase está nas letras e nas batidas de fundo, o RMS pode ser moderado, com os vocais destacados e uma base rítmica presente.

No rock, com suas guitarras distorcidas e bateria intensa, o RMS pode ser alto, refletindo a energia e a intensidade do estilo.

No samba, um estilo musical repleto de ritmo e percussão, o RMS pode ser moderado a alto, devido à presença de instrumentos percussivos com amplitudes significativas.

No sertanejo, com sua instrumentação que inclui violão e acordeão, o RMS pode variar dependendo do arranjo e do ritmo da música.

### Força do ataque (onset strength)

A força do ataque é uma medida que indica a intensidade ou saliência de mudanças de atividade rítmica no sinal de áudio.

**Exemplos em gêneros musicais:**

No forró, um estilo musical animado e festivo, a força do ataque pode ser alta e marcante, especialmente quando se trata da entrada de instrumentos como o acordeão e a zabumba.

No funk, com suas batidas fortes e impactantes, a força do ataque é geralmente alta e constante ao longo da música, destacando a presença das batidas de bateria e das linhas de baixo.

No jazz, um gênero musical com muita improvisação e nuances, a força do ataque pode variar dependendo do momento da música. Em partes mais calmas e suaves, como um solo de saxofone, a força do ataque pode ser baixa. Já em momentos mais intensos, como um solo de bateria, a força do ataque pode ser alta e pulsante.

No rap, onde a voz e a percussão têm destaque, a força do ataque pode ser elevada, enfatizando o impacto das batidas e dos vocais.

No rock, com suas guitarras distorcidas e bateria enérgica, a força do ataque tende a ser alta, refletindo a intensidade e a agressividade características desse estilo.

No samba, um gênero musical com forte presença de instrumentos percussivos, a força do ataque pode ser bastante marcante, enfatizando o ritmo contagiante das batucadas.

No sertanejo, com sua instrumentação que inclui violão e instrumentos de cordas, a força do ataque pode variar dependendo da intensidade da música e da ênfase nos momentos de início de frase ou refrão.

### Contraste espectral de um sinal

O contraste espectral é uma medida que representa a diferença ou o contraste entre os componentes de frequência em um sinal de áudio. Ele avalia a variação da energia sonora em diferentes bandas de frequência, permitindo identificar diferenças de timbre e textura sonora.

**Exemplos em gêneros musicais:**

No forró, um estilo musical tradicionalmente composto por acordeões, zabumbas e triângulos, o contraste espectral pode ser moderado. Isso ocorre devido à presença marcante das frequências médias, relacionadas aos acordeões, e das frequências baixas, relacionadas às percussões.

No funk, que tem uma ênfase nas batidas de baixo e bateria, o contraste espectral pode ser alto. Isso ocorre porque há uma diferença acentuada de energia entre as frequências graves do baixo e as frequências mais agudas dos elementos percussivos.

No jazz, com sua variedade de instrumentos e improvisação, o contraste espectral pode ser médio a alto. Os instrumentos de sopro, como saxofone e trompete, geralmente têm uma distribuição de energia mais pronunciada em frequências médias e altas, resultando em um contraste maior com as frequências baixas dos instrumentos de base, como piano e baixo.

No rap, onde a voz e a batida são elementos centrais, o contraste espectral pode ser moderado a alto. A voz humana geralmente ocupa uma faixa de frequências médias a altas, enquanto a batida de fundo pode abranger um espectro mais amplo, incluindo frequências graves e agudas.

No rock, com suas guitarras distorcidas e bateria potente, o contraste espectral tende a ser alto. As frequências graves e agudas das guitarras combinadas com a bateria enérgica criam uma distribuição de energia contrastante em diferentes faixas de frequência.

No samba, um gênero com forte presença de percussão, o contraste espectral pode ser moderado. As diferentes percussões, como tamborim, surdo e pandeiro, têm distribuições de energia distintas em diferentes faixas de frequência, resultando em um contraste perceptível.

No sertanejo, com sua instrumentação de violões e instrumentos de corda, o contraste espectral pode ser médio a alto. Os violões geralmente possuem uma energia significativa nas frequências médias, enquanto os instrumentos de corda podem ter uma distribuição mais equilibrada entre as frequências baixas e altas.

### Largura de banda espectral

A largura de banda espectral é uma medida que indica a extensão ou o intervalo de frequências presentes em um sinal de áudio. Ela representa a largura da faixa de frequências em que a maior parte da energia sonora está concentrada.

**Exemplos em gêneros musicais:**

No forró, que é caracterizado por instrumentos como acordeões, zabumbas e triângulos, a largura de banda espectral pode ser moderada. Isso ocorre porque a maioria das energias sonoras está concentrada em um intervalo específico de frequências, relacionado aos instrumentos mencionados.

No funk, com sua ênfase nas batidas de baixo e bateria, a largura de banda espectral pode ser ampla. Isso significa que a energia sonora se espalha por uma ampla faixa de frequências, abrangendo as frequências graves do baixo e as frequências agudas dos elementos percussivos.

No jazz, que possui uma variedade de instrumentos e improvisação, a largura de banda espectral pode ser ampla. Os instrumentos de sopro, como saxofone e trompete, podem abranger uma ampla faixa de frequências, enquanto outros instrumentos como piano e baixo contribuem para uma distribuição equilibrada em diferentes frequências.

No rap, onde a voz e a batida são elementos centrais, a largura de banda espectral pode ser moderada. A voz humana geralmente se concentra em frequências médias a altas, enquanto a batida de fundo pode abranger uma faixa de frequências mais ampla, mas ainda com predominância nas médias e agudas.

No rock, com suas guitarras distorcidas e bateria potente, a largura de banda espectral tende a ser ampla. As guitarras podem cobrir uma ampla gama de frequências, desde as graves até as agudas, enquanto a bateria contribui com uma distribuição de energia significativa em várias faixas de frequência.

No samba, um gênero com forte presença de percussão, a largura de banda espectral pode ser moderada. Cada instrumento de percussão, como tamborim, surdo e pandeiro, contribui com uma faixa de frequências específica, resultando em uma distribuição equilibrada em diferentes faixas de frequência.

No sertanejo, a largura de banda espectral pode variar dependendo da abordagem musical específica. Em arranjos mais tradicionais, a largura de banda pode ser mais moderada, destacando-se as frequências vocais e dos instrumentos de corda, como o violão. Já em arranjos mais modernos, podem ser adicionados elementos eletrônicos e uma maior variação de frequências.

### Nivelamento espectral

O nivelamento espectral, também conhecido como equalização, é um processo que ajusta a intensidade de diferentes frequências em um sinal de áudio. Ele é usado para equilibrar o volume ou a ênfase de certas partes do espectro de frequência, tornando o som mais balanceado e agradável aos ouvidos.

**Exemplos em gêneros musicais:**

No forró, o nivelamento espectral pode enfatizar as frequências médias e altas, destacando os instrumentos melódicos, como o acordeão e o triângulo, que têm uma presença forte nessas faixas de frequência.

No funk, o nivelamento espectral pode enfatizar as frequências graves, como os sons de baixo e as batidas de bateria, que são elementos essenciais nesse estilo.

No jazz, o nivelamento espectral pode ser mais equilibrado, destacando tanto as frequências graves dos instrumentos de base, como o contrabaixo, quanto as frequências médias e altas dos instrumentos melódicos, como saxofone e piano.

No rap, o nivelamento espectral pode ser usado para enfatizar as frequências médias e agudas da voz do rapper, permitindo que as letras e o fluxo se destaquem.

No rock, o nivelamento espectral pode ser usado para enfatizar as frequências médias e altas das guitarras distorcidas e da bateria, criando um som poderoso e agressivo.

No samba, o nivelamento espectral pode ser mais equilibrado, destacando as diferentes camadas rítmicas dos instrumentos de percussão e as frequências médias das vozes dos cantores.

No sertanejo, o nivelamento espectral pode ser usado para enfatizar as frequências médias e altas dos violões e das vozes, proporcionando clareza e brilho ao som.

Guardando as características em um arquivo CSV:

In [11]:
df_features.to_csv('./all_features.csv', index=False)